In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, RationalQuadratic as RQ
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

## Initialization

In [11]:
collision_model = pickle.load(open('Poly_2degree.sav', 'rb'))

samples_cpt = 100

diam_leg_statesn = 10
diam_leg_range = [0.8,2] 

diam_brace_statesn = 10 
diam_brace_range = [0.4,0.8]

boat_speed_statesn = 10 
boat_speed_range = [0.1,6]

angle_statesn = 10 
angle_range = [0,45] 

penetration_statesn = 10 
penetration_range = [0,6] 

conditional = np.zeros((diam_leg_statesn*diam_brace_statesn*boat_speed_statesn*angle_statesn,penetration_statesn))

C:\Users\user\Anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


## Discretization of the state space

In [12]:
diam_leg_state_step = (diam_leg_range[1]-diam_leg_range[0])/diam_leg_statesn
diam_leg_states = np.arange(diam_leg_range[0], diam_leg_range[1]+diam_leg_state_step, diam_leg_state_step)

diam_brace_state_step = (diam_brace_range[1]-diam_brace_range[0])/diam_brace_statesn
diam_brace_states = np.arange(diam_brace_range[0], diam_brace_range[1]+diam_brace_state_step, diam_brace_state_step)

boat_speed_state_step = (boat_speed_range[1]-boat_speed_range[0])/boat_speed_statesn
boat_speed_states = np.arange(boat_speed_range[0], boat_speed_range[1]+boat_speed_state_step, boat_speed_state_step)

angle_state_step = (angle_range[1]-angle_range[0])/angle_statesn
angle_states = np.arange(angle_range[0], angle_range[1]+angle_state_step, angle_state_step)

penetration_state_step = (penetration_range[1]-penetration_range[0])/penetration_statesn
penetration_states = np.arange(penetration_range[0], penetration_range[1]+penetration_state_step, penetration_state_step)

print(diam_leg_states, diam_brace_states, boat_speed_states, angle_states, penetration_states)

[0.8  0.92 1.04 1.16 1.28 1.4  1.52 1.64 1.76 1.88 2.   2.12] [0.4  0.44 0.48 0.52 0.56 0.6  0.64 0.68 0.72 0.76 0.8  0.84] [0.1  0.69 1.28 1.87 2.46 3.05 3.64 4.23 4.82 5.41 6.  ] [ 0.   4.5  9.  13.5 18.  22.5 27.  31.5 36.  40.5 45. ] [0.  0.6 1.2 1.8 2.4 3.  3.6 4.2 4.8 5.4 6. ]


## Computation of the conditional probability table (CPT)

In [13]:
index_cpt = 0

for i in range(diam_leg_statesn):
    diam_leg_input = np.random.uniform(diam_leg_states[i], diam_leg_states[i+1], samples_cpt)
    for j in range(diam_brace_statesn):
        diam_brace_input = np.random.uniform(diam_brace_states[j], diam_brace_states[j+1], samples_cpt)
        for k in range(boat_speed_statesn):
            boat_speed_input = np.random.uniform(boat_speed_states[k], boat_speed_states[k+1], samples_cpt)
            for l in range(angle_statesn):
                angle_input = np.random.uniform(angle_states[l], angle_states[l+1], samples_cpt)
                
                #Computation of all samples via the polynomial model:
                penetration_input = np.zeros((samples_cpt, 4))
                penetration_input[:,0] = angle_input
                penetration_input[:,1] = boat_speed_input
                penetration_input[:,2] = diam_leg_input
                penetration_input[:,3] = diam_brace_input
                poly = PolynomialFeatures(2)
                poly_features = poly.fit_transform(penetration_input)
                penetration_output = collision_model.predict(poly_features)
                
                #Checking the probability of each output state
                for m in range(penetration_statesn):
                    conditional[index_cpt ,m] = ((penetration_states[m] < penetration_output) & \
                     (penetration_output < penetration_states[m+1])).sum() / samples_cpt
                    
                if (penetration_output < 0).sum() > 0 : # All penetrations < 0 are allocated to state 0
                    conditional[index_cpt ,0] = conditional[index_cpt , 0] + 1 - np.sum(conditional[index_cpt, :])                          
                    
                ## Cross-checking the CPT ###    
#                 if (penetration_output < -0.1).sum() > 0 : 
#                         print(i,j,k,l,penetration_output)
                
                index_cpt += 1

In [15]:
conditional.shape

(10000, 10)

## Prior distributions

### Uniform distribution

In [5]:
v_boat_in = np.ones(boat_speed_statesn)/boat_speed_statesn
v_boat_input = np.tile ( np.repeat(v_boat_in,angle_statesn), diam_leg_statesn*diam_brace_statesn)
v_boat = np.transpose(np.tile(v_boat_input, (penetration_statesn, 1) ) )

angle_in = np.ones(angle_statesn)/angle_statesn
angle_input = np.tile(angle_in, diam_leg_statesn*diam_brace_statesn*boat_speed_statesn)
angle = np.transpose(np.tile(angle_input, (penetration_statesn, 1) ) )

## Marginalizing out random variables

In [6]:
joint = conditional*v_boat*angle

output = np.zeros((diam_leg_statesn*diam_brace_statesn, penetration_statesn))
index_out = 0

for i in range(diam_leg_statesn):
    for j in range(diam_brace_statesn):
        output[index_out, :] = np.sum(joint[index_out*boat_speed_statesn*angle_statesn:(index_out+1)*boat_speed_statesn*angle_statesn, :], axis=0 )
        index_out += 1

In [7]:
minor_repair = -10
major_repair = -500
repair_threshold = 3
consequence_penetration = np.zeros(penetration_statesn)
consequence_penetration[penetration_states[:-1] <= repair_threshold] = minor_repair
consequence_penetration[penetration_states[:-1] > repair_threshold] = major_repair
consequence_penetration

array([ -10.,  -10.,  -10.,  -10.,  -10.,  -10., -500., -500., -500.,
       -500.])

## Consequences and risk estimation

In [8]:
cost_leg_range = [-10, -110]
cost_brace_range = [-5, -55]
minor_repair = -10
major_repair = -500
repair_threshold = 3

cost_leg = np.linspace(cost_leg_range[0], cost_leg_range[1], diam_leg_statesn)
cost_leg_rep = np.repeat(cost_leg, diam_brace_statesn)

cost_brace = np.linspace(cost_brace_range[0], cost_brace_range[1], diam_brace_statesn)
cost_brace_rep = np.tile(cost_brace, diam_leg_statesn)

consequence_penetration = np.zeros(penetration_statesn)
consequence_penetration[penetration_states[:-1] <= repair_threshold] = minor_repair
consequence_penetration[penetration_states[:-1] > repair_threshold] = major_repair

consequence = np.tile( consequence_penetration, (diam_leg_statesn*diam_brace_statesn,1) )

risk = output*(consequence)
# designs = np.sum(risk, axis=1) + cost_leg_rep + cost_brace_rep 
designs = np.reshape( np.sum(risk, axis=1) + cost_leg_rep + cost_brace_rep, (diam_leg_statesn, diam_brace_statesn) )

In [66]:
designs

array([[-133.29      , -148.15555556, -147.34111111, -155.83666667,
        -159.43222222, -160.08777778, -172.01333333, -175.11888889,
        -177.24444444, -205.34      ],
       [-123.33111111, -124.47666667, -134.44222222, -134.11777778,
        -137.71333333, -145.22888889, -148.33444444, -162.71      ,
        -161.89555556, -165.98111111],
       [-103.08222222, -105.69777778, -112.23333333, -118.76888889,
        -120.40444444, -128.41      , -141.31555556, -135.60111111,
        -145.56666667, -162.88222222],
       [ -77.44333333,  -85.93888889,  -91.00444444, -109.3       ,
         -96.72555556, -116.49111111, -132.33666667, -117.80222222,
        -131.68777778, -130.38333333],
       [ -72.38444444,  -76.96      ,  -83.49555556,  -86.11111111,
         -97.05666667,  -97.71222222, -102.77777778, -108.82333333,
        -114.86888889, -119.93444444],
       [ -80.55555556,  -86.11111111,  -91.66666667,  -97.22222222,
        -102.77777778, -108.33333333, -113.88888889, -119

## Optimum design

In [9]:
max_index = np.unravel_index( np.argmax(designs), designs.shape )
print(diam_leg_states[max_index[0]], diam_leg_states[max_index[0]+1])
print(diam_brace_states[max_index[1]], diam_brace_states[max_index[1]+1])

1.28 1.4
0.4 0.44


## Additional settings

In [9]:
import sys
np.set_printoptions(threshold=sys.maxsize)
conditional

array([[0.9, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.1, 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.1, 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.3, 0.7, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.3, 0.7, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.3, 0.7, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. 